# Sentiment analysis of financial data using a large language model (LLM)

Document a large language model (LLM) specialized in sentiment analysis for financial news using the ValidMind Library. 

This interactive notebook shows you how to set up the ValidMind Library, initializes the library, and uses a specific prompt template for analyzing the sentiment of sentences in a dataset. The notebook also includes example data to test the model's ability to correctly identify sentiment as positive, negative, or neutral.

::: {.content-hidden when-format="html"}
## Contents    
- [About ValidMind](#toc1__)    
- [Before you begin](#toc2__)    
- [Setting up](#toc3__)    
  - [Install the ValidMind Library](#toc3_1__)    
  - [Initialize the ValidMind Library](#toc3_2__)    
    - [Register sample model](#toc3_2_1__)    
    - [Apply documentation template](#toc3_2_2__)    
    - [Get your code snippet](#toc3_2_3__)    
  - [Preview the documentation template](#toc3_3__)    
- [Get ready to run the analysis](#toc4__)    
- [Get your sample dataset ready for analysis](#toc5__)    
- [Run the model documentation tests](#toc6__)    
- [Next steps](#toc7__)    
- [Upgrade ValidMind](#toc8__)    

:::
<!-- jn-toc-notebook-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/jn-toc-notebook-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a id='toc1__'></a>

## About ValidMind

ValidMind's suite of tools enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Library to automate documentation and validation tests, and then use the ValidMind Platform to collaborate on documentation initiatives. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, we recommend going through the following resources first:

- [Get started](https://docs.validmind.ai/get-started/get-started.html) — The basics, including key concepts, and how our products work
- [ValidMind Library](https://docs.validmind.ai/developer/validmind-library.html) — The path for developers, more code samples, and our developer reference

<a id='toc2__'></a>

## Before you begin

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>For access to all features available in this notebook, you'll need access to a ValidMind account.</b></span>
<br></br>
<a href="https://docs.validmind.ai/guide/configuration/register-with-validmind.html" style="color: #DE257E;"><b>Register with ValidMind</b></a></div>

This notebook requires an OpenAI API secret key to run. If you don't have one, visit [API keys](https://platform.openai.com/account/api-keys) on OpenAI's site to create a new key for yourself. Note that API usage charges may apply.

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

<a id='toc3__'></a>

## Setting up

<a id='toc3_1__'></a>

### Install the ValidMind Library

To install the library:

In [ ]:
%pip install -q validmind

<a id='toc3_2__'></a>

### Initialize the ValidMind Library

<a id='toc3_2_1__'></a>

#### Register sample model

Let's first register a sample model for use with this notebook:

1. In a browser, [log in to ValidMind](https://docs.validmind.ai/guide/configuration/log-in-to-validmind.html).

2. In the left sidebar, navigate to **Inventory** and click **+ Register Model**.

3. Enter the model details and click **Next >** to continue to assignment of model stakeholders. ([Need more help?](https://docs.validmind.ai/guide/model-inventory/register-models-in-inventory.html))

   For example, to register a model for use with this notebook, select the following use case: `Marketing/Sales - Analytics`

4. Select your own name under the **MODEL OWNER** drop-down.

5. Click **Register Model** to add the model to your inventory.

<a id='toc3_2_2__'></a>

#### Apply documentation template

Once you've registered your model, let's select a documentation template. A template predefines sections for your model documentation and provides a general outline to follow, making the documentation process much easier.

1. In the left sidebar that appears for your model, click **Documents** and select **Documentation**.

2. Under **TEMPLATE**, select `LLM-based Text Classification`.

3. Click **Use Template** to apply the template.

<a id='toc3_2_3__'></a>

#### Get your code snippet

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the ValidMind Library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

1. On the left sidebar that appears for your model, select **Getting Started** and click **Copy snippet to clipboard**.
2. Next, [load your model identifier credentials from an `.env` file](https://docs.validmind.ai/developer/model-documentation/store-credentials-in-env-file.html) or replace the placeholder with your own code snippet:

In [ ]:
# Load your model identifier credentials from an `.env` file

%load_ext dotenv
%dotenv .env

# Or replace with your code snippet

import validmind as vm

vm.init(
    # api_host="...",
    # api_key="...",
    # api_secret="...",
    # model="...",
)

<a id='toc3_3__'></a>

### Preview the documentation template

Let's verify that you have connected the ValidMind Library to the ValidMind Platform and that the appropriate *template* is selected for your model.

You will upload documentation and test results unique to your model based on this template later on. For now, **take a look at the default structure that the template provides with [the `vm.preview_template()` function](https://docs.validmind.ai/validmind/validmind.html#preview_template)** from the ValidMind library and note the empty sections:

In [ ]:
vm.preview_template()

<a id='toc4__'></a>

## Get ready to run the analysis

Import the ValidMind `FoundationModel` and `Prompt` classes needed for the sentiment analysis later on:

In [ ]:
from validmind.models import FoundationModel, Prompt

Check your access to the OpenAI API:

In [ ]:
import os

import dotenv

dotenv.load_dotenv()

if os.getenv("OPENAI_API_KEY") is None:
    raise Exception("OPENAI_API_KEY not found")

In [ ]:
from openai import OpenAI

model = OpenAI()


def call_model(prompt):
    return (
        model.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt},
            ],
        )
        .choices[0]
        .message.content
    )

Set the prompt guidelines for the sentiment analysis:

In [ ]:
prompt_template = """
You are an AI with expertise in sentiment analysis, particularly in the context of financial news.
Your task is to analyze the sentiment of a specific sentence provided below.
Before proceeding, take a moment to understand the context and nuances of the financial terminology used in the sentence.

Sentence to Analyze:
```
{Sentence}
```

Please respond with the sentiment of the sentence denoted by one of either 'positive', 'negative', or 'neutral'.
Please respond only with the sentiment enum value. Do not include any other text in your response.

Note: Ensure that your analysis is based on the content of the sentence and not on external information or assumptions.
""".strip()

prompt_variables = ["Sentence"]

<a id='toc5__'></a>

## Get your sample dataset ready for analysis

To perform the sentiment analysis for financial news we're going to load a local copy of this dataset: https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news.

This dataset contains two columns, `Sentiment` and `Sentence`. The sentiment can be `negative`, `neutral` or `positive`.

In [ ]:
import pandas as pd

df = pd.read_csv("./datasets/sentiments_with_predictions.csv")

<a id='toc6__'></a>

## Run the model documentation tests

First, use the ValidMind Library to initialize the dataset and model objects necessary for documentation. The ValidMind `predict_fn` function allows the model to be tested and evaluated in a standardized manner:

In [ ]:
vm_test_ds = vm.init_dataset(
    dataset=df,
    input_id="test_dataset",
    text_column="Sentence",
    target_column="Sentiment",
)

vm_model = vm.init_model(
    model=FoundationModel(
        predict_fn=call_model,
        prompt=Prompt(
            template=prompt_template,
            variables=prompt_variables,
        ),
    ),
    input_id="gpt_35_model",
)

# Assign model predictions to the test dataset
vm_test_ds.assign_predictions(vm_model, prediction_column="gpt_35_prediction")

Next, use the ValidMind Library to run validation tests on the model. The `vm.run_documentation_tests` function analyzes the current model's documentation template and collects all the tests associated with it into a test suite.

The function then runs the test suite, logs the results to the ValidMind API and displays them to you.

In [ ]:
test_suite = vm.run_documentation_tests(
    inputs={
        "dataset": vm_test_ds,
        "model": vm_model,
    }
)

<a id='toc7__'></a>

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way: view the prompt validation test results as part of your model documentation in the ValidMind Platform:

1. In the ValidMind Platform, click **Documentation** under Documents for the model you registered earlier. ([Need more help?](https://docs.validmind.ai/guide/model-documentation/working-with-model-documentation.html))

2. Expand **2. Data Preparation** or **3. Model Development** to review all test results.

What you can see now is a more easily consumable version of the prompt validation testing you just performed, along with other parts of your model documentation that still need to be completed.

If you want to learn more about where you are in the model documentation process, take a look our documentation on the [ValidMind Library](https://docs.validmind.ai/developer/validmind-library.html).

<a id='toc8__'></a>

## Upgrade ValidMind

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;">After installing ValidMind, you’ll want to periodically make sure you are on the latest version to access any new features and other enhancements.</div>

Retrieve the information for the currently installed version of ValidMind:

In [ ]:
%pip show validmind

If the version returned is lower than the version indicated in our [production open-source code](https://github.com/validmind/validmind-library/blob/prod/validmind/__version__.py), restart your notebook and run:

```bash
%pip install --upgrade validmind
```

You may need to restart your kernel after running the upgrade package for changes to be applied.